In [ ]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
from pathlib import Path

In [ ]:
print("Tensorflow version: ", tf.__version__)

In [ ]:
# Path to the data directory
data_dir = Path("dataset/")

# Get list of all the images
images = list(data_dir.glob("*.png")),

print("Number of images: ", len(images))

In [ ]:
# Examples 

_, ax = plt.subplots(2, 2, figsize=(6,4))

for i in range(4):
    img = mpimg.imread(str(images[i]))
    ax[i//2, i%2].imshow(img)
    ax[i//2, i%2].axis('off')
plt.show()